In [1]:
!pip install binance_historical_data mplfinance stable-baselines3 backtesting tensorboard binance ta


[notice] A new release of pip is available: 23.2.1 -> 23.3.1
[notice] To update, run: pip install --upgrade pip


In [4]:
import numpy as np
import pandas as pd
from util import OHLC_COLUMNS, add_features, download_data
from util import preprocess

df = download_data()

dataset = df.loc[:, OHLC_COLUMNS].astype(np.float64)
dataset = pd.DataFrame(dataset.to_numpy(), columns=OHLC_COLUMNS)

dataset = add_features(dataset)
preprocessed_dataset, scaler = preprocess(dataset)

,Open,High,Low,Close,SMA_256,SMA_512,SMA_1024,MACD_diff,RSI,CCI,ADX,stoch
1023,1.1284,1.1293,1.1284,1.1293,0.027575,-0.002864,-0.033380,-0.000692,66.150065,61.021979,51.960560,74.742268
1024,1.1284,1.1309,1.1284,1.1309,0.029032,-0.001275,-0.031812,-0.000656,67.418785,66.758116,51.217777,75.912409
1025,1.1310,1.1361,1.1310,1.1361,0.033741,0.003445,-0.027268,-0.000347,71.197206,104.147986,51.111836,100.000000
1026,1.1362,1.1364,1.1337,1.1364,0.033990,0.003865,-0.026914,-0.000196,71.403251,119.346483,51.044391,100.000000
1027,1.1375,1.1382,1.1356,1.1376,0.035056,0.005082,-0.025857,-0.000094,72.258137,140.769151,51.169446,96.610169
...,...,...,...,...,...,...,...,...,...,...,...,...
1600991,1.3340,1.3350,1.3330,1.3340,0.015722,0.024636,0.027428,0.000066,56.646180,80.341880,37.715651,55.555556
1600992,1.3350,1.3350,1.3330,1.3340,0.015643,0.024522,0.027410,0.000033,56.646180,73.483842,37.648823,55.555556
1600993,1.3340,1.3340,1.3320,1.3320,0.014048,0.022879,0.025853,-0.000128,50.217128,-1.899335,36.916354,33.333333
1600994,1.3330,1.3330,1.3320,1.3320,0.013976,0.022775,0.025834,-0.000233,50.217128,-24.691358,36.236204,33.333333


In [6]:
from util import test_preprocess_invert_preprocess, test_orig_val

test_preprocess_invert_preprocess(dataset)
test_orig_val(dataset)

Mean Absolute Error for Open: 1.107702642891245
Mean Absolute Error for High: 1.3524301038831599
Mean Absolute Error for Low: 0.7988906677050388
Mean Absolute Error for Close: 1.1182623099180922
Mean Absolute Error for SMA_256: 7.725245422375353e-18
Mean Absolute Error for SMA_512: 7.540261310804373e-18
Mean Absolute Error for SMA_1024: 6.0620489304659e-18
Mean Absolute Error for MACD_diff: 7.061264968818638e-18
Mean Absolute Error for RSI: 4.1411930450655985e-17
Mean Absolute Error for CCI: 2.7459341034366415e-15
Mean Absolute Error for ADX: 8.666418617795355e-16
Mean Absolute Error for stoch: 3.4714785398373443e-16
Average MAE: 0.36477381036646167


In [8]:
# from google.colab import drive
# drive.mount('/content/drive')
# 
# import time
# import os
# try:
#   os.makedirs("./drive/MyDrive/colab/model-weights/")
# except:
#   pass
# my_model.save(f"./drive/MyDrive/colab/model-weights/model-weights-{time.time()}.pt")

In [9]:
%load_ext tensorboard
%tensorboard --logdir tensorboard

Launching TensorBoard...

In [11]:
from model import TransformerExtractor
from env import CustomEnv
from stable_baselines3.common.vec_env import SubprocVecEnv
from stable_baselines3 import PPO
from stable_baselines3.common.env_util import make_vec_env
import torch

from stable_baselines3.common.env_checker import check_env

env = CustomEnv(dataset, scaler)
check_env(env)
del env

env = make_vec_env(lambda: CustomEnv(dataset, scaler), n_envs=10, seed=42, vec_env_cls=SubprocVecEnv)
# env = make_vec_env(lambda: CustomEnv(dataset, scaler), n_envs=3, seed=42, vec_env_cls=SubprocVecEnv)

# policy_kvargs = dict(activation_fn=torch.nn.LeakyReLU,
#                      features_extractor_class=LSTMExtractor,
#                      features_extractor_kwargs=dict(lstm_hidden_size=128, lstm_layers=2),
#                      net_arch=[256, 256, 256])
policy_kvargs = dict(activation_fn=torch.nn.LeakyReLU,
                     features_extractor_class=TransformerExtractor,
                     features_extractor_kwargs=dict(transformer_hidden_size=128, transformer_heads=2,
                                                    transformer_layers=2),
                     net_arch=[256, 256, 256])
# {'gamma': 0.8, 'ent_coef': 0.02, 'gae_lambda': 0.92}
rl_model = PPO("MultiInputPolicy", env,
               verbose=1,
               tensorboard_log="./tensorboard/",
               ent_coef=0.02,
               gae_lambda=0.92,
               gamma=0.8,
               policy_kwargs=policy_kvargs)
# rl_model.set_parameters("rl-model-best-model/best_model/best_model.zip")

obs length = {'prices_sequence': (256, 12), 'other': (2,)}


/mnt/storage/dima/Doc/Documents/Особисте/Проєкти/Projects/price_prediction_experiment/venv/lib/python3.11/site-packages/stable_baselines3/common/env_checker.py:244: UserWarning: Your observation prices_sequence has an unconventional shape (neither an image, nor a 1D vector). We recommend you to flatten the observation to have only a 1D vector or use a custom policy to properly process the data.
  warnings.warn(


Using cuda device


In [12]:
rl_model.policy

MultiInputActorCriticPolicy(
  (features_extractor): TransformerExtractor(
    (extractors): ModuleDict(
      (other): Flatten(start_dim=1, end_dim=-1)
      (prices_sequence): TransformerEncoder(
        (layers): ModuleList(
          (0-1): 2 x TransformerEncoderLayer(
            (self_attn): MultiheadAttention(
              (out_proj): NonDynamicallyQuantizableLinear(in_features=12, out_features=12, bias=True)
            )
            (linear1): Linear(in_features=12, out_features=128, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
            (linear2): Linear(in_features=128, out_features=12, bias=True)
            (norm1): LayerNorm((12,), eps=1e-05, elementwise_affine=True)
            (norm2): LayerNorm((12,), eps=1e-05, elementwise_affine=True)
            (dropout1): Dropout(p=0.1, inplace=False)
            (dropout2): Dropout(p=0.1, inplace=False)
          )
        )
      )
    )
  )
  (pi_features_extractor): TransformerExtractor(
    (extractors):

In [13]:
from stable_baselines3.common.callbacks import CheckpointCallback, EvalCallback

checkpoint_callback = CheckpointCallback(
    save_freq=50_000,
    save_path="rl-model-checkpoints/",
    name_prefix="rl_model",
    verbose=1,
    save_vecnormalize=True,
    save_replay_buffer=True
)
eval_env = make_vec_env(lambda: CustomEnv(dataset, scaler, episode_length=2048), n_envs=1, seed=42,
                        vec_env_cls=SubprocVecEnv)
eval_callback = EvalCallback(eval_env, best_model_save_path="rl-model-best-model/best_model",
                             log_path="rl-model-best-model/results", eval_freq=50_000, verbose=1,
                             n_eval_episodes=5)

rl_model.learn(total_timesteps=1_000_000, callback=[checkpoint_callback, eval_callback])

Logging to ./tensorboard/PPO_59
---------------------------------
| rollout/           |          |
|    ep_len_mean     | 1.02e+03 |
|    ep_rew_mean     | -1.71    |
| time/              |          |
|    fps             | 235      |
|    iterations      | 1        |
|    time_elapsed    | 87       |
|    total_timesteps | 20480    |
---------------------------------
----------------------------------------
| rollout/                |            |
|    ep_len_mean          | 1.02e+03   |
|    ep_rew_mean          | -1.71      |
| time/                   |            |
|    fps                  | 148        |
|    iterations           | 2          |
|    time_elapsed         | 276        |
|    total_timesteps      | 40960      |
| train/                  |            |
|    approx_kl            | 0.02419753 |
|    clip_fraction        | 0.248      |
|    clip_range           | 0.2        |
|    entropy_loss         | -1.08      |
|    explained_variance   | -8.36      |
|    learning

In [14]:
evaluations = np.load("rl-model-best-model/results/evaluations.npz")
evaluations_df = pd.DataFrame()
evaluations_df["timesteps"] = evaluations["timesteps"]
evaluations_df["results"] = evaluations["results"].mean(axis=1)
evaluations_df

,timesteps,results
0,500000,0.0
1,1000000,0.0


In [15]:
from util import SKIP_STEPS, calculate_observation
# backtesting

from backtesting import Backtest, Strategy, set_bokeh_output


class NeuralNetStrat(Strategy):
    def __init__(self, broker, data, params):
        super().__init__(broker, data, params)
        self.buy_price = None
        self.expected_gain = None

    def init(self):
        pass

    def next(self):
        if len(self.data) > SKIP_STEPS:
            df = self.data.df.iloc[-SKIP_STEPS:].copy()
            df.drop(columns=["Volume"], inplace=True)
            observation, curr_close, _ = calculate_observation(df, scaler, self.buy_price)

            action, _ = rl_model.predict(observation, deterministic=True)
            if self.buy_price is None:
                if action == 1:
                    self.buy()
                    print(f"buy")
                    self.buy_price = curr_close
            else:
                if action == 2:
                    commission = 0.001
                    sell_fee = curr_close * (1 - commission)
                    buy_fee = self.buy_price * (1 + commission)
                    gain_from_trade_fee = (sell_fee - buy_fee) / buy_fee
                    print(f"gain {gain_from_trade_fee}")
                    self.sell()
                    self.buy_price = None


backtest_dataset = dataset[-20000:]
backtest_dataset["Volume"] = df[1023:]["Volume"]
bt = Backtest(backtest_dataset, NeuralNetStrat, commission=.001,
              exclusive_orders=True)
stats = bt.run()
stats

/mnt/storage/dima/Doc/Documents/Особисте/Проєкти/Projects/price_prediction_experiment/venv/lib/python3.11/site-packages/backtesting/_plotting.py:50: UserWarning: Jupyter Notebook detected. Setting Bokeh output to notebook. This may not work in Jupyter clients without JavaScript support (e.g. PyCharm, Spyder IDE). Reset with `backtesting.set_bokeh_output(notebook=False)`.
  warnings.warn('Jupyter Notebook detected. '


Loading BokehJS ... 
 
 Warning: Requesting CDN BokehJS version '3.3.0rc2' from local development version '3.3.0rc2+2.g273f1258'. This configuration is unsupported and may not work!

/tmp/ipykernel_37403/1128459029.py:39: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  backtest_dataset["Volume"] = df[1023:]["Volume"]
/mnt/storage/dima/Doc/Documents/Особисте/Проєкти/Projects/price_prediction_experiment/venv/lib/python3.11/site-packages/backtesting/backtesting.py:1054: FutureWarning: Index.is_numeric is deprecated. Use pandas.api.types.is_any_real_numeric_dtype instead
  (data.index.is_numeric() and
/tmp/ipykernel_37403/1128459029.py:40: UserWarning: Data index is not datetime. Assuming simple periods, but `pd.DateTimeIndex` is advised.
  bt = Backtest(backtest_dataset, NeuralNetStrat, commission=.001,


Start                               1580996.0
End                                 1600995.0
Duration                              19999.0
Exposure Time [%]                         0.0
Equity Final [$]                      10000.0
Equity Peak [$]                       10000.0
Return [%]                                0.0
Buy & Hold Return [%]               32.405567
Return (Ann.) [%]                         0.0
Volatility (Ann.) [%]                     NaN
Sharpe Ratio                              NaN
Sortino Ratio                             NaN
Calmar Ratio                              NaN
Max. Drawdown [%]                        -0.0
Avg. Drawdown [%]                         NaN
Max. Drawdown Duration                    NaN
Avg. Drawdown Duration                    NaN
# Trades                                  0.0
Win Rate [%]                              NaN
Best Trade [%]                            NaN
Worst Trade [%]                           NaN
Avg. Trade [%]                    

In [16]:
set_bokeh_output(notebook=False)
bt.plot()

INFO:bokeh.io.state:Session output file 'NeuralNetStrat.html' already exists, will be overwritten.
/mnt/storage/dima/Doc/Documents/Особисте/Проєкти/Projects/price_prediction_experiment/venv/lib/python3.11/site-packages/backtesting/_plotting.py:659: UserWarning: found multiple competing values for 'toolbar.active_drag' property; using the latest value
  fig = gridplot(
/mnt/storage/dima/Doc/Documents/Особисте/Проєкти/Projects/price_prediction_experiment/venv/lib/python3.11/site-packages/backtesting/_plotting.py:659: UserWarning: found multiple competing values for 'toolbar.active_scroll' property; using the latest value
  fig = gridplot(


GridPlot(id='p1121', ...)